In [1]:
import akasha
import os
# import PyPDF2
from time import time
import requests
from bs4 import BeautifulSoup

/home/ychm0304/anaconda3/envs/aks-plus/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = "openai:gpt-4o"
result = {}
contexts = []
report = {}
response = ""

In [3]:
QA = akasha.Doc_QA(model=model, max_doc_len=8000)
summary = akasha.Summary(chunk_size=1000, max_doc_len=7000)

In [4]:
system_prompt = "你是一個非常擅長寫報告的電池產業專業人員，你會根據指定的格式來生成報告，且內容會完全符合指定的主題。若你無法回答會告訴我原因。"
titles = {
    "全球電池市場概況": [
        "市場規模與增長趨勢",
        "主要市場區域分析",
        "主要企業概況",
    ], 
    "技術發展趨勢": [
        "鋰離子電池技術進展",
        "固態電池技術",
        "其他新興電池技術",
    ], 
    "應用領域與市場機會": [
        "電動車市場",
        "消費電子產品",
        "能源儲存系統",
        "工業及其他應用",
        "市場需求預測",
        "市場挑戰與機遇",
    ], 
    "政策與供應鏈分析": [
        "各國政策支持與法規要求",
        "環保與回收政策",
        "供應鏈與原材料供應",
        "產業佈局與投資趨勢",
    ]
}

links = [
    "https://www.maonline.com.tw/article_inside.php?i=798",
    "https://www.materialsnet.com.tw/DocView.aspx?id=53579",
    "https://ic.tpex.org.tw/introduce.php?ic=E000",
    "https://ieknet.iek.org.tw/ieknews/news_open.aspx?actiontype=ieknews&indu_idno=5&nsl_id=1039fffa4a7d408cb3f5c579762637e1",
    "https://yez.one/post/batterycell",
    "https://www.thenewslens.com/article/181546"
]

theme = "電池產業的發展趨勢"
# titles = ["企業前瞻"]

In [5]:
start_time = time()

def generate_report(theme, titles, links):
    for title, subtitle in titles.items():
        format_prompt = f"以{theme}為主題，請你總結撰寫出與\"{title}\"相關的內容，其中需包含{subtitle}，不需要結論，不需要回應要求。"
        for link in links:
            print("##################################")
            print("Processing link: ", link)
            print("##################################")

            try:
                response = requests.get(link)
                response.raise_for_status()  
                soup = BeautifulSoup(response.content, 'html.parser')
                texts = soup.get_text()
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
                texts = '' 

            contexts.append(
                summary.summarize_articles(
                    articles=texts,
                    format_prompt=format_prompt,
                )
            )

        response = QA.ask_self(
            prompt=f"將此內容以客觀角度進行融合，避免使用\"報告中提到\"相關詞彙，避免修改專有名詞，避免做出總結，\
                        直接撰寫內容，避免回應要求。",
            info=contexts,
            model="openai:gpt-4"
        )
        result[title] = response
        contexts = []
            
        print(f"----------integrated summary of {title}----------")
        print(response + "\n")
    return result


result = generate_report(theme, titles, links)
previous_result = ""
for value in result.values():
    previous_result += value

result["內容摘要"] = summary.summarize_articles(
    articles=previous_result,
    format_prompt=f"將內容以{theme}為主題進行摘要，將用字換句話說，意思不變，不需要結論，不需要回應要求。",
    summary_len=500
)

total_time = time() - start_time

##################################
Processing link:  https://www.maonline.com.tw/article_inside.php?i=798
##################################


Reduce_map Summary: 100%|██████████| 1/1 [00:04<00:00,  4.78s/it]


根據您提供的資訊，以下是與全球電池市場概況相關的內容：

市場規模與增長趨勢:
- 鋰電池市場在未來五年內將保持快速成長，並且預估到2030年，鋰電池將主導整個電池市場。
- 全球電池市場的快速成長主要受到電動車和可再生能源等領域的需求推動。

主要市場區域分析:
- 臺灣的鋰電池供應鏈與全球市場發展趨勢一致，但產品發展速度已經落後，需要盡快趕上才能在市場上競爭。
- 臺灣三大集團投入巨資興建電池芯廠，預計2025年全產能運轉後可達到9.5GWh約當量，但仍存在原材料供應問題和產能不足的挑戰。

主要企業概況:
- 臺灣三大集團在鋰電池市場上投入巨資興建電池芯廠，以應對全球市場的需求，但仍需面對原材料供應問題和產能不足的挑戰。

以上是與全球電池市場概況相關的內容摘要，希望對您有所幫助。 




##################################
Processing link:  https://www.materialsnet.com.tw/DocView.aspx?id=53579
##################################


Reduce_map Summary: 100%|██████████| 1/1 [00:07<00:00,  7.22s/it]


根據B FORTUNE BUSINESS INSIGHTS的觀察分析，預估液流電池在儲能技術中的市場規模將從2022年的108.8億美元增加至2029年的312.0億美元，複合年增長率達16.3 %。在論壇中，液流電池與鋰電池在儲能時間成本的比較顯示，液流電池在使用超過5~8小時儲能時間時具有競爭潛力，並且具有安全性優勢。

此外，根據Voltstorage公司的白皮書，長時間儲能是未來重要的技術，尤其是對於再生能源的穩定供應。美國能源部預期2040年美國儲能功率的日間較長持續時間比例將達60%，而儲能需求時間延後，預估使用儲能技術中，50 %採用液流電池。

在主要市場區域分析方面，歐洲液流電池(FBE)目前有25家公司參與，並致力於推進液流電池在歐洲的研發和商業化部署。此外，美國能源部預期2040年美國儲能功率的日間較長持續時間比例將達60%，而儲能需求時間延後，預估使用儲能技術中，50 %採用液流電池。

在主要企業概況方面，液流電池產業的發展趨勢受到了關注，特別是在能源安全挑戰增加、氣候變遷議題加速全球推動淨零排放計畫的情況下，可再生能源產業的推升對液流電池的關注逐漸升溫。 




##################################
Processing link:  https://ic.tpex.org.tw/introduce.php?ic=E000
##################################


...

在電池產業的發展趨勢中，各國政策支持與法規要求扮演著重要角色。許多國家紛紛推出支持電池產業發展的政策，包括補助、稅收優惠、研發資金等，以鼓勵企業投入電池相關技術研發和生產。此外，許多國家也制定了相應的法規要求，以確保電池產品的安全性、環保性和品質標準。

環保與回收政策也是電池產業發展中不可忽視的一環。許多國家都加強了對電池回收利用的管理與監管，並推動電池回收再利用技術的研發和應用，以減少對環境的影響，並實現循環經濟的目標。

供應鏈與原材料供應是電池產業中的關鍵問題。隨著電動車和儲能系統的快速發展，對鋰、鎳、鈷等原材料的需求急劇增加，這也帶動了原材料供應鏈的調整和優化。各國紛紛加大對鋰礦等關鍵原材料的開發力度，並積極推動產業間的合作，以確保原材料供應的穩定性和可持續性。

產業佈局與投資趨勢也在不斷演變。許多國家紛紛加大對電池產業的投資，並積極引導企業進行產業佈局的優化和升級，以提高產業的競爭力和可持續發展能力。同時，國際間的產業合作與投資也日益活躍，為電池產業的全球化發展注入了新的動力。

以上是與政策與供應鏈分析相關的內容，希望對您有所幫助。 




##################################
Processing link:  https://www.thenewslens.com/article/181546
##################################


Reduce_map Summary: 100%|██████████| 1/1 [00:04<00:00,  4.10s/it]


根據這篇文章，我們可以看到與政策與供應鏈分析相關的內容。首先，各國政策支持與法規要求方面，臺灣政府積極支持電池產業發展，並提出相關發展計畫，以鼓勵業者投資和創新。同時，中國政府也在電池產業中佔有領先地位，其政策支持對全球電池供應鏈產生重大影響。

在環保與回收政策方面，全球對環境保護意識的提高促使各國政府加強對電池回收和再利用的法規要求，這對電池供應鏈和產業佈局產生了影響。

供應鏈與原材料供應方面，臺灣業者在材料創新應用方面具有競爭優勢，這對於全球鋰電池產業的供應鏈具有關鍵地位。同時，臺灣業者與中國業者合作，共同創造多元化電池芯的解決方案，這反映了供應鏈的跨國合作趨勢。

最後，在產業佈局與投資趨勢方面，預測電動車市場將持續增長，這對電池產業的產業佈局和投資趨勢產生了影響。臺灣各大集團也積極發展電池產業，這顯示了產業佈局和投資趨勢的積極樂觀態度。

總的來說，政策支持、環保與回收政策、供應鏈與原材料供應、產業佈局與投資趨勢是影響電池產業發展的重要因素，也是各國業者需要關注和應對的挑戰和機遇。 




----------integrated summary of 政策與供應鏈分析----------
鋰電池產業的發展趨勢中，政策與供應鏈分析扮演著重要角色。各國政策支持與法規要求對於鋰電池產業的發展至關重要。許多國家紛紛推出補助措施、稅收優惠和環保法規，以鼓勵鋰電池產業的發展。例如，中國政府積極推動新能源汽車發展，並提出一系列支持政策，包括補助和配套基礎設施建設。歐洲則致力於推動可再生能源和電動車，並訂立了相應的排放標準和環保法規。

環保與回收政策也對鋰電池產業產生深遠影響。隨著對可持續發展的重視，各國紛紛加強對鋰電池的回收和再利用政策。這不僅影響著鋰電池的生產和設計，也對供應鏈和原材料供應產生影響。產業必須思考如何建立可持續的供應鏈，並開發環保友好的回收技術。

供應鏈與原材料供應是鋰電池產業中的關鍵議題。鋰、鎳、鋁等原材料的供應穩定性和價格波動直接影響著產業的發展。產業必須密切關注原材料供應鏈的動態變化，並積極尋求多元化的供應來源，以降低風險。

產業佈局與投資趨勢也是鋰電池產業的重要議題。隨著市場需求的增長，許多企業紛紛擴大產能、進行技術研發和佈局全球市場。臺灣作為鋰電池產業的重要參與者，必須密切關注全球產業佈局和投資趨勢，並積極參與全球競爭。



Reduce_map Summary: 100%|██████████| 1/1 [00:08<00:00,  8.02s/it]


全球電池市場正在迅速擴大，主要受到電動車和可再生能源等領域需求的推動。未來五年內，鋰電池市場將持續快速增長，預計到2030年，鋰電池將成為主導市場的產品。液流電池在儲能技術中的市場規模也將持續擴大。亞太地區是全球電池市場的主要區域，擁有龐大的電動車市場和儲能系統需求。知名公司如東芝、LG化學、三星SDI、CATL等在鋰電池等領域具有領先地位。新興技術如固態電池、鋁空氣電池、鋰硫電池也受到關注和研究。電池產業在電動車、消費電子產品、能源儲存系統、工業及其他領域都有著巨大的市場機會。然而，市場挑戰包括技術創新、成本壓力、原材料供應、安全性問題和回收利用等。政策支持與法規要求對於鋰電池產業的發展至關重要，並且環保與回收政策也對產業產生深遠影響。供應鏈與原材料供應、產業佈局與投資趨勢也是重要議題。 






In [9]:
result["內容摘要"] = summary.summarize_articles(
    articles=previous_result,
    format_prompt=f"將內容以{theme}為主題進行摘要，將用字換句話說，意思不便，不需要結論，不需要回應要求。",
    summary_len=500
)

Reduce_map Summary:   0%|          | 0/1 [00:00<?, ?it/s]

Reduce_map Summary: 100%|██████████| 1/1 [00:09<00:00,  9.20s/it]


全球電池市場正在迅速增長，尤其是鋰電池市場，預計在2022年至2030年間將達到4000億美元的市值，年增率達33%。這主要是由於電動車、儲能系統和消費電子產品的需求增加，尤其是在中國、歐洲和美國等主要成長區域。然而，臺灣的鋰電池供應鏈面臨技術不成熟、市場規模小和原材料供應等挑戰。臺灣的三大集團正在投入大量資金興建電池芯廠，但需要加快發展速度以應對全球市場需求。

另一方面，液流電池產業在能源轉型和氣候變遷的大背景下具有巨大的發展潛力。根據美國能源部的預估，液流電池在儲能技術中的應用比例預計將達到50%。全球電池市場的競爭格局主要由幾家主要企業主導，包括特斯拉、東芝、LG化學、三星SDI、寧德時代等。鋰離子電池技術在未來五年內將保持快速成長，並且未來十年內有望主宰整個電池市場。固態電池技術被認為是未來的發展方向之一，具有高安全性、高能量密度和長壽命等優點。除了鋰離子電池和固態電池，還有其他新興電池技術如鋰硫電池、鋰空氣電池等，這些新技術在能量密度、成本和環保性能上都有著潛在優勢，有望成為未來電池技術的重要發展方向。

許多國家都積極推動能源轉型，並且制定了支持電池產業發展的政策，例如補助、稅收優惠和研發資助等。這些政策鼓勵了電池產業的成長和創新。供應鏈與原材料供應是電池產業不可或缺的一環，原材料的穩定供應和成本控制對於企業的競爭力至關重要。產業佈局與投資趨勢則受到各國政策、市場需求和技術發展的影響，企業應該密切關注各國政策變化和原材料供應情況，並根據市場趨勢和法規要求調整產業佈局和投資策略。 






In [6]:
print("----------Complete report----------")
print("Total time: ", total_time, "s")
for key, value in result.items():
    print(key, ":\n\n\t", value, "\n")

----------Complete report----------
Total time:  503.20611810684204 s
全球電池市場概況 :

	 全球電池市場正處於快速增長階段，主要受到電動車和可再生能源等領域的需求推動。鋰電池市場在未來五年內將保持快速成長，並且預估到2030年，鋰電池將主導整個電池市場。此外，液流電池在儲能技術中的市場規模將從2022年的108.8億美元增加至2029年的312.0億美元，複合年增長率達16.3 %。

在主要市場區域分析方面，亞太地區是全球電池市場的主要區域，其中包括中國、日本和韓國等國家。這些地區擁有龐大的電動車市場和儲能系統需求，因此成為全球電池市場的主要推動力。臺灣的鋰電池供應鏈與全球市場發展趨勢一致，但產品發展速度已經落後，需要盡快趕上才能在市場上競爭。

在主要企業概況方面，全球電池市場的主要企業包括東芝、LG化學、三星SDI、CATL等知名公司。這些企業在鋰電池等電池技術領域具有領先地位，並且積極參與電動車和儲能系統領域的競爭。臺灣三大集團在鋰電池市場上投入巨資興建電池芯廠，以應對全球市場的需求，但仍需面對原材料供應問題和產能不足的挑戰。 

技術發展趨勢 :

	 鋰離子電池技術在市場上的重要性不斷提升，預估未來幾年將保持快速增長。這種技術的進步主要集中在提高能量密度、延長循環壽命和降低成本。新的陽極和電解質材料的開發，以及對電池充放電過程的更好理解，都有助於提高鋰離子電池的性能。

固態電池技術是近年來受到極大關注的新興技術。固態電池被認為具有更高的安全性和能量密度，並且可能成為未來電池技術的主流。許多公司和研究機構都在積極開發固態電池技術，並且已經取得了一些重要的突破，但仍然需要克服製造成本和製程問題。

除了固態電池技術外，還有許多其他新興電池技術也值得關注。例如，鋁空氣電池、鋰硫電池、鈉離子電池等技術都在不同程度上受到關注和研究。這些新興技術可能在未來對電池市場產生重大影響，並且可能成為未來的發展趨勢之一。

液流電池在能源安全和氣候變遷議題下受到關注，並且預估市場規模將持續增長。液流電池在超過5~8小時儲能時間具有競爭潛力。 

應用領域與市場機會 :

	 電池產業在多個應用領域中都有著巨大的市場機會。在電動車市場，隨著對清潔能源交通工具的需求增加，鋰電池將持續成為主要動力電池技術

In [ ]:
generate_report_tool = akasha.akashas.create_tool(
    "Generate Report",
    """
    Generate a report based on a given theme, titles, and links.

    Parameters:
    - theme (str): The main theme of the report
    - titles (Dict[str, str]): A dictionary of titles and subtitles
    - links (List[str]): A list of links to process

    Returns:
    - Dict[str, str]: A dictionary containing each title and its corresponding content
    """,
    func = generate_report
)

titles_input_tool = akasha.akashas.create_tool(
    "Titles Input",
    """
    Input the titles and subtitles of the report.

    Parameters:
    - theme (str): The main theme of the report

    Returns:
    - Dict[str, str]: A dictionary of titles and subtitles
    """,
    func = lambda theme: titles
)

In [7]:
part = input("想要修改哪一部分的內容? (輸入標題或\"結束\")\n")
while part != "結束":
    if part in result:
        print(f"----------目前內容----------\n\n{result[part]}\n\n------------------------")
        new_prompt = input("想要如何修改:\n")
        
    else:
        print("找不到對應的標題")
    part = input("想要修改哪一部分的內容? (輸入標題或\"結束\")\n")

----------目前內容----------

全球電池市場正在迅速擴大，主要受到電動車和可再生能源等領域需求的推動。未來五年內，鋰電池市場將持續快速增長，預計到2030年，鋰電池將成為主導市場的產品。液流電池在儲能技術中的市場規模也將持續擴大。亞太地區是全球電池市場的主要區域，擁有龐大的電動車市場和儲能系統需求。知名公司如東芝、LG化學、三星SDI、CATL等在鋰電池等領域具有領先地位。新興技術如固態電池、鋁空氣電池、鋰硫電池也受到關注和研究。電池產業在電動車、消費電子產品、能源儲存系統、工業及其他領域都有著巨大的市場機會。然而，市場挑戰包括技術創新、成本壓力、原材料供應、安全性問題和回收利用等。政策支持與法規要求對於鋰電池產業的發展至關重要，並且環保與回收政策也對產業產生深遠影響。供應鏈與原材料供應、產業佈局與投資趨勢也是重要議題。

------------------------
找不到對應的標題
找不到對應的標題
